In [ ]:
# Import the packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [ ]:
# Import custom functions
from helpers import *

# Dataset treatment

We realized that some values must be treated again :
- The sizes must be scaled because women are generally smaller than men
- The box office revenues must be scaled because of the inflation
- We need to associate a region to a country for spatial analysis
- We have to reduce the number of genres by doing some clustering